<a href="https://colab.research.google.com/github/josh-english/DataVisualization/blob/master/Midterm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1 - Data Preprocessing

## Description

Preprocess 20 Newsgroup dataset as corpus and visualize its statistical information. (10’)

## Get the Data

In [1]:
# get data from gensim
import gensim.downloader as api

print(api.load("20-newsgroups", return_path=False))
PATH = "/root/gensim-data/20-newsgroups/20-newsgroups.gz"

# unzip data to pandas dataframe
import pandas as pd
df = pd.read_json(PATH, lines=True, compression="gzip")

print(df.shape)
print(df[0:5])

[==================================================] 100.1% 13.8/13.8MB downloaded
<20-newsgroups.Dataset object at 0x7fa2610cb7b8>
(18846, 4)
                                                data     id    set  \
0  From: db7n+@andrew.cmu.edu (D. Andrew Byler)\n...  21408  train   
1  From: mpaul@unl.edu (marxhausen paul)\nSubject...  21388  train   
2  From: eggertj@moses.atc.ll.mit.edu (Jim Eggert...  20600  train   
3  From: tas@pegasus.com (Len Howard)\nSubject: R...  20820  train   
4  From: db7n+@andrew.cmu.edu (D. Andrew Byler)\n...  21335  train   

                    topic  
0  soc.religion.christian  
1  soc.religion.christian  
2  soc.religion.christian  
3  soc.religion.christian  
4  soc.religion.christian  


## Alternatively

In [2]:
# from: https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html
from sklearn.datasets import fetch_20newsgroups
twenty_train = fetch_20newsgroups(subset='all', shuffle=True, random_state=42)

twenty_train.target_names
len(twenty_train.data)
len(twenty_train.filenames)

print("\n".join(twenty_train.data[0].split("\n")[:3]))
print("\n" + twenty_train.target_names[twenty_train.target[0]])

From: Mamatha Devineni Ratnam <mr47+@andrew.cmu.edu>
Subject: Pens fans reactions
Organization: Post Office, Carnegie Mellon, Pittsburgh, PA

rec.sport.hockey


# Step 2 - Vocabularies from BoW and TF-IDF

## Description

Build two different vocabularies upon different preprocessing ways; Learn Bag-of-words (BoW) and TF-IDF model with each vocabulary accordingly. (10’)

# Step 3 - 2 LDA Models

## Description

Train two LDA models upon the vocabularies in Step 2; Visualize topics with four different methods; and eventually get the topic distribution (as feature) for each document. (20’)

# Step 4 - 2 Doc2Vec Models

## Decsription

Train two Doc2Vec models upon the vocabularies in Step 2; Visualize your leaned word and document embedding space; Collect Doc2Vec representation of each document. (20’)

# Step 5 - Clustering (BoW, TF-IDF, Topic Distribution, and Doc2Vec)

## Decsription

Conduct document clustering by K-means with four different doc. representations: 1) BoW; 2) TF-IDF; 3) Topics distribution; and 4) Doc2Vec. Compare different results by Normalized Mutual Information (NMI) and visualize the clustering results. (20’)

# Step 6 - Analysis

## Description

Do experiment analysis from the following aspects: 1) Impact of different preprocessing ways (e.g., how to filter vocabulary; using n-gram model); 2) Impact of different topic numbers; and 3) Different training methods for Doc2Vec; 4) What’s the key factor for doc. visualization? (20’)

# Step 7 - Presentation

## Description

Every group is required to give a presentation with slides in our class. The presentation time is about 5-10 minutes. Each talk should include the following contents.
1) Introduction to your group members and team assignments.
2) A clear illustration for your project workflow.
3) All the experimental results you have obtained with some necessary experimental analyses.
4) A live demo for your visualization result. For example, a demo for visualizing different topics, or
displaying your word/doc embedding space. (Jupyter Notebook is recommended.)

# Step 8 - Report

## Description

The final submission is required as follows.
1) A two-page pdf report including 1) a brief introduction to the project and your method; 2) all the
necessary results and analyses; 3) references for the tools and papers you used in this work.
2) A package file including all your source codes and visualization results.